# Solver Pre-Compilation
Most of the time, using the 'algorithm =' argument within model.run() is the best practice for assigning a particular algorithm to be used when a particular model is ran. This however will create issues when the model is ran within a loop body as each iteration will take almost a full five seconds in make alone, dramatically increasing the time taken to run each iteration. This is why it is best to precompile any model within a loop body. Below is a quick example of how to do so. 

## Setup the Environment

In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../..')))

In [2]:
import time

MatPlotLib is used for creating custom visualizations

In [3]:
import matplotlib.pyplot as plt

In [4]:
import gillespy2
from gillespy2 import TauHybridCSolver

## Build the Model

In [5]:
def build_michaelis_menten(parameter_values=None):
    # Initialize Model
    model = gillespy2.Model(name="Michaelis_Menten")

    # Define Species
    A = gillespy2.Species(name='Substrate', initial_value=301)
    B = gillespy2.Species(name='Enzyme', initial_value=120)
    C = gillespy2.Species(name='Enzyme_Substrate_Complex', initial_value=0)
    D = gillespy2.Species(name='Product', initial_value=0)
    
    # Add Species to Model
    model.add_species([A, B, C, D])

    # Define Parameters
    rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
    rate2 = gillespy2.Parameter(name='rate2', expression= 0.5)
    rate3 = gillespy2.Parameter(name='rate3', expression = 0.1)
    
    # Add Parameters to Model
    model.add_parameter([rate1, rate2, rate3])

    # Define Reactions
    r1 = gillespy2.Reaction(name="r1",reactants={A:1, B:1}, products={C:1}, rate=rate1)
    r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1, B:1}, rate=rate2)
    r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1, D:1}, rate=rate3)
    
    # Add Reactions to Model
    model.add_reaction([r1, r2, r3])
    
    # Define Timespan
    tspan = gillespy2.TimeSpan.linspace(t=100, num_points=101)
    
    # Add Timespan to Model
    model.timespan(tspan)
    return model

### Instantiate the Model

In [6]:
model = build_michaelis_menten()

### Instantiate the Solver
Call `get_best_solver_algo` on the `model` object and pass the desired algorithm name like you would with `algorithm=`. Also pass the `model` object in a seperate bracket. This will Precompile the model and solver if the solver is a C++ solver.

In [7]:
precomp_solver = model.get_best_solver_algo("Tau-Hybrid")(model=model)

## Runtime Comparison: Non-Precompiled Solver vs Precompiled Solver
Below is an example of why pre-compilation is neccsary when running a model in a loop body using a C++ solver.

### Non-Precompiled Solver

In [8]:
np_start = time.perf_counter()
for i in range(20):
    np_results = model.run(solver=TauHybridCSolver)
np_end = time.perf_counter()
np_time = np_end - np_start

print(f"Time to complete 20 runs without using pre compiled model and solver: {np_time}")

Time to complete 20 runs without using pre compiled model and solver: 114.47748833300001


### Precompiled Solver

In [9]:
solver = TauHybridCSolver(model=model)

p_start = time.perf_counter()
for i in range(20):
    p_results = model.run(solver=solver)
p_end = time.perf_counter()
p_time = p_end - p_start

print(f"Time to complete 20 runs using pre compiled model and solver: {p_time}")

Time to complete 20 runs using pre compiled model and solver: 0.2220953750000092
